# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import re
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
immigration_data = pd.read_csv(
    '../data/immigration_data_sample.csv'
)
immigration_data.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [10]:
immigration_data.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [4]:
i_94_map = {
    'cicid': 'uid',
    'i94yr': 'year',
    'i94mon': 'month',
    'i94cit': 'city',
    'i94res': 'res',
    'i94port': 'port',
    'i94mode': 'mode',
    'i94addr': 'address',
    'i94bir': 'bir',
    'i94visa': 'visa'
}

i_94_df = immigration_data[
    i_94_map.keys()
].rename(
    columns=i_94_map
)

i_94_df.head()

,uid,year,month,city,res,port,mode,address,bir,visa
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,1.0,HI,61.0,2.0
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,1.0,TX,26.0,2.0
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,1.0,FL,76.0,2.0
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,1.0,CA,25.0,2.0
4,985523.0,2016.0,4.0,111.0,111.0,CHM,3.0,NY,19.0,2.0


In [5]:
immigration_data.nunique()

Unnamed: 0    1000
cicid         1000
i94yr            1
i94mon           1
i94cit          88
i94res          91
i94port         70
arrdate         30
i94mode          4
i94addr         51
depdate        109
i94bir          85
i94visa          3
count            1
dtadfile        39
visapost        97
occup            3
entdepa          9
entdepd         10
entdepu          0
matflag          1
biryear         85
dtaddto         99
gender           3
insnum          29
airline        101
admnum        1000
fltno          502
visatype        10
dtype: int64

In [6]:
immigration_data.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [8]:
demographics_data = pd.read_csv(
    '../data/us-cities-demographics.csv',
    delimiter=';'
)
demographics_data.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Wichita,Kansas,34.6,192354.0,197601.0,389955,23978.0,40270.0,2.56,KS,American Indian and Alaska Native,8791
1,Allen,Pennsylvania,33.5,60626.0,59581.0,120207,5691.0,19652.0,2.67,PA,Black or African-American,22304
2,Danbury,Connecticut,37.3,43435.0,41227.0,84662,3752.0,25675.0,2.74,CT,Black or African-American,8454
3,Nashville,Tennessee,34.1,314231.0,340365.0,654596,27942.0,88193.0,2.39,TN,Hispanic or Latino,67526
4,Stamford,Connecticut,35.4,64941.0,63936.0,128877,2269.0,44003.0,2.70,CT,Asian,11013


In [13]:
for col in demographics_data.columns:
    print(f'- `{col}`:')

- `City`:
- `State`:
- `Median Age`:
- `Male Population`:
- `Female Population`:
- `Total Population`:
- `Number of Veterans`:
- `Foreign-born`:
- `Average Household Size`:
- `State Code`:
- `Race`:
- `Count`:


In [9]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


ModuleNotFoundError: No module named 'pyspark'

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.